In [1]:
from google.colab import files
uploaded = files.upload()

Saving charity_data.csv to charity_data.csv


In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df= application_df.drop(columns=['EIN', 'NAME'])


In [4]:
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [6]:
# Look at APPLICATION_TYPE value counts for binning
application_type= application_df['APPLICATION_TYPE'].value_counts()
application_type

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_type[application_type<500].index)
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
classification= application_df['CLASSIFICATION'].value_counts()
classification

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification[classification<1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies = pd.get_dummies(application_df)
application_dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [11]:
# Split our preprocessed data into our features and target arrays
X = application_dummies.drop('IS_SUCCESSFUL', axis=1)
y = application_dummies['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test= train_test_split(X,y, random_state=1)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
X_train_scaled.shape

(25724, 43)

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = X_train_scaled.shape[1]
hidden_node1 = 76
hidden_node2 = 16
hidden_node3= 21
hidden_node4= 11
hidden_node5= 26

nn_model = tf.keras.models.Sequential()
# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_node1, activation='relu', input_dim=input_features))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_node2, activation='relu'))
nn_model.add(tf.keras.layers.Dense(units=hidden_node3, activation='sigmoid'))
nn_model.add(tf.keras.layers.Dense(units=hidden_node4, activation='sigmoid'))
nn_model.add(tf.keras.layers.Dense(units=hidden_node5, activation='sigmoid'))
# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 76)                3344      
                                                                 
 dense_1 (Dense)             (None, 16)                1232      
                                                                 
 dense_2 (Dense)             (None, 21)                357       
                                                                 
 dense_3 (Dense)             (None, 11)                242       
                                                                 
 dense_4 (Dense)             (None, 26)                312       
                                                                 
 dense_5 (Dense)             (None, 1)                 27        
                                                                 
Total params: 5,514
Trainable params: 5,514
Non-trainabl

In [15]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
#Create a callback that saves the model's weights every five epochs
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define checkpoints variables
os.makedirs('checkpoints/', exist_ok=True)
checkpoint_path= 'checkpoints/weights.{epoch}.hdf5'

# Create callback 
cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                              verbose=1,
                              save_weights_only=True,
                              save_freq='epoch',
                              period=5)

In [17]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50, callbacks=[cp_callback])

Epoch 1/50
804/804 [==============================] - 4s 3ms/step - loss: 0.6007 - accuracy: 0.6965
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5621 - accuracy: 0.7299
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5550 - accuracy: 0.7312
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5528 - accuracy: 0.7307
Epoch 5/50
795/804 [============================>.] - ETA: 0s - loss: 0.5513 - accuracy: 0.7324
Epoch 5: saving model to checkpoints/weights.5.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5512 - accuracy: 0.7325
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5496 - accuracy: 0.7331
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5490 - accuracy: 0.7340
Epoch 8/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5482 - accuracy: 0.7332
Epoch 9/50
804/804 [==============================] - 2s 2ms/st

In [18]:
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5537 - accuracy: 0.7285 - 522ms/epoch - 2ms/step
Loss: 0.5537236928939819, Accuracy: 0.7285131216049194


In [19]:
#Hyper paramenter
#Install keras-tuner
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.1/172.1 KB 2.8 MB/s eta 0:00:00


In [20]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model_hp = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model_hp.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value= 90,
        step=5), activation=activation, input_dim=input_features))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model_hp.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=30,
            step=5),
            activation=activation))
    
    nn_model_hp.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model_hp.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model_hp

In [21]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=50,
    hyperband_iterations=2)

In [ ]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=50,validation_data=(X_test_scaled,y_test))

Trial 162 Complete [00h 00m 43s]
val_accuracy: 0.7306122183799744

Best val_accuracy So Far: 0.73399418592453
Total elapsed time: 01h 10m 37s

Search: Running Trial #163

Value             |Best Value So Far |Hyperparameter
sigmoid           |sigmoid           |activation
36                |71                |first_units
5                 |4                 |num_layers
1                 |6                 |units_0
6                 |26                |units_1
21                |16                |units_2
16                |16                |units_3
1                 |1                 |units_4
50                |17                |tuner/epochs
17                |6                 |tuner/initial_epoch
2                 |3                 |tuner/bracket
2                 |2                 |tuner/round
0157              |0035              |tuner/trial_id

Epoch 18/50
804/804 [==============================] - 5s 4ms/step - loss: 0.5777 - accuracy: 0.7301 - val_loss: 0.5765 - val_accurac

In [23]:
# Best model hyperparameters 
top_hp = tuner.get_best_hyperparameters(1)
for param in top_hp:
  print(param.values)

{'activation': 'sigmoid', 'first_units': 71, 'num_layers': 4, 'units_0': 6, 'units_1': 26, 'units_2': 16, 'units_3': 16, 'tuner/epochs': 17, 'tuner/initial_epoch': 6, 'tuner/bracket': 3, 'tuner/round': 2, 'tuner/trial_id': '0035', 'units_4': 1}


In [24]:
# Best model
top_model = tuner.get_best_models(1)
for model in top_model:
  model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
  print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

268/268 - 1s - loss: 0.5728 - accuracy: 0.7340 - 1s/epoch - 4ms/step
Loss: 0.5727969408035278, Accuracy: 0.73399418592453


In [25]:
# Get best model hyperparameters
best_hp = tuner.get_best_hyperparameters()[0]
best_hp.values

{'activation': 'sigmoid',
 'first_units': 71,
 'num_layers': 4,
 'units_0': 6,
 'units_1': 26,
 'units_2': 16,
 'units_3': 16,
 'tuner/epochs': 17,
 'tuner/initial_epoch': 6,
 'tuner/bracket': 3,
 'tuner/round': 2,
 'tuner/trial_id': '0035',
 'units_4': 1}

In [26]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5728 - accuracy: 0.7340 - 714ms/epoch - 3ms/step
Loss: 0.5727969408035278, Accuracy: 0.73399418592453


In [27]:
best_model.save('.../AlphabetSoupCharity_Optimization.h5')